## DASHBOARD

- VISUAL TOOL TO COMPARE RENTS IN AIRBNB AND IDEALISTA. 

    - Choose neighbourhood and number of rooms (measure of size) and the app will display the estimated gross income and the average availability for that area.
    
- VISUALIZATION OF MAJOR DIFFERENCES BETWEEN RESIDENTIAL AND VACATIONAL MARKETS IN MADRID

In [1]:
import pandas as pd
import numpy as np
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import plotly.plotly as py
from plotly.graph_objs import *


In [3]:
center=pd.read_csv('center.csv')

# Remove accents. Plotly Dash app does not work with accents
center['neighborhood'] = center.neighborhood.str.replace('ü','u')
center['neighborhood'] = center.neighborhood.str.replace('á','a')
center['neighborhood'] = center.neighborhood.str.replace('é','e')
center['neighborhood'] = center.neighborhood.str.replace('í','i')
center['neighborhood'] = center.neighborhood.str.replace('ó','o')
center['neighborhood'] = center.neighborhood.str.replace('ú','u')

In [4]:
prem= center.groupby('neighborhood',as_index=False).Holyday_rental_premium_pct.mean().sort_values(by='Holyday_rental_premium_pct', ascending=True)
neig=center.groupby('neighborhood',as_index=False).availability_30.mean()
offer = center.groupby('neighborhood')[['Number_flats_IDEALISTA','Number_flats_AIRBNB']].sum().reset_index()
offer = offer.reset_index()

In [5]:
%cd ~/Master-IV/PROYECTO_FM/SHP_ETRS89/

/home/dsc/Master-IV/PROYECTO_FM/SHP_ETRS89


### DASHBOARD APP

In [58]:
rom = center['rooms']
neighborhood = center['neighborhood'].unique()
rooms=center.rooms.sort_values(ascending=True)
rooms=rooms.unique()


app=dash.Dash()
app.layout = html.Div([
      html.Div([
          html.Div([
            html.H1("MADRID RENTAL MARKET 2017",className='seven columns',style={'text-align': 'center',
                                                                              'fontSize':25,'color':'#3e4444',
                                                                              'font-family':'Arial, sans-serif',
                                                                               'boxshadow':'#C0C0C0'})],
    className='banner'),
    html.Hr(style={'margin': '0', 'margin-bottom': 5}),
    
    html.Div([
          html.Div([
               html.H2('Select Neighbourhood',style={'fontSize':15,'color':'#7f7f7f'}),
                dcc.Dropdown(
                    id='zonas',
                    options=[{'label': i, 'value': i} for i in neighborhood],
                    value='Acacias'),
        ],
        style={'width': '40%', 'display': 'inline-block'}),

        html.Div([
             html.H2('Select Rooms',style={'fontSize':15,'color':'#7f7f7f'}),
                dcc.Dropdown(
                    id='rom',
                    options=[{'label': i, 'value': i} for i in rooms],
                    value=rom),
        ], style={'width': '40%', 'float': 'right', 'display': 'inline-block'}),
    ]),
    html.Div([
         html.H3(style={'margin-bottom': 10}),
        dcc.Graph(id='barras'),
    ],style={'width': '40%', 'display': 'inline-block'}),
    html.Div([
         html.H4(style={'margin-bottom': 10}),
        dcc.Graph(
            id='ava'),
            ], style={'width': '40%', 'display': 'inline-block','float':'right'}),
    html.Div([
        html.H4(style={'margin-bottom': 10}),
        dcc.Graph(id='Premium',
                  figure={"data":[go.Bar(y=prem['Holyday_rental_premium_pct'],x=prem['neighborhood'],width=0.3,                                      
                                         name='Percentage',
                                         marker=dict(color= '#ff8080', line = dict(
                                         color = '#f9ccac',
                                         width = 0.8)),
                                         opacity=0.6)],
                         "layout":go.Layout(title='AIRBNB RENT PREMIUM',
                                            xaxis=dict(dtick=1, title='Neighbourhood',tickangle=45),
                                            yaxis=dict(title='% Gain over Idealista rents'))},
                                             style={"width":"94vw",
                                                    "height":"90vh",
                                                   'margin-bottom':30,
                                                   'margin-left':5,
                                                   'margin-right':5}),
    ]),
    html.Div([
        dcc.Graph(id='offer',
                 figure={'data':[go.Bar(y=offer['Number_flats_AIRBNB'],x=offer['neighborhood'],width=0.3,                                      
                                         name='Listings in Airbnb',
                                         marker=dict(color= '#ff8080', line = dict(
                                         color = '#f9ccac',
                                         width = 0.8))),
                                go.Bar(y=offer['Number_flats_IDEALISTA'],x=offer['neighborhood'],width=0.3,                                      
                                         name='Listings in Idealista',
                                         marker=dict(color= '#b3ff66', line = dict(
                                         color = '#b3ff66',
                                         width = 0.8)))],
                         "layout":go.Layout(title='RENTAL MARKET OFFER',
                                            xaxis=dict(dtick=1, title='Neighbourhood',tickangle=45),
                                            yaxis=dict(title='Number of listings'))},
                                             style={"width":"94vw",
                                                     "height":"90vh",
                                                   'margin-bottom':15,
                                                   'margin-left':5,
                                                   'margin-right':5}),
    ]),
      ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
         'margin-left': 'auto',
        'margin-right': 'auto'})
    
])

@app.callback(dash.dependencies.Output('barras','figure'),
              [dash.dependencies.Input('zonas','value'),
               dash.dependencies.Input('rom','value')])

def upgrade_chart(zonas,rom):
    dff=center[center.rooms==rom]
    dff_filt=dff[dff.neighborhood==zonas]
    return {'data':[go.Bar(x=dff.Estimated_income_Airbnb_EUR, y=dff_filt['neighborhood'], width=0.20,
                name='Airbnb',
                orientation='h',
                marker=dict(color= '#ff8080', line = dict(
                color = '#ff8080',
                width = 0.2))),
                   go.Bar(x=dff.Monthly_rent_Idealista_EUR, y=dff_filt['neighborhood'], width=0.20,
                name='Idealista',
                orientation='h',
                marker=dict(color= '#b3ff66', line = dict(
                color = '#b3ff66',
                width = 0.2)))],
           'layout':go.Layout(title='Estimated Gross Monthly Income',
                              xaxis={'title': 'Rent EUR'},
                              yaxis={'title': 'Neighbourhood'},
                              width=500, 
                              height=300)}
    

@app.callback(dash.dependencies.Output('ava','figure'),
              [dash.dependencies.Input('zonas','value')])

def new_chart(zonas):
    dava=center[center.neighborhood==zonas]
    return {'data':[go.Bar(y=dava['availability_30'],x=dava['rooms'],width=0.25,
                name='Airbnb', 
                marker=dict(color= '#ff8080',line = dict(
                color = '#ff8080',
                width = 0.2)))],
           'layout':go.Layout(title='Airbnb Average availability in a Month',
                              xaxis={'title': 'Rooms'},
                              yaxis={'title': 'Days'},
                              width=500, 
                              height=300)}   



app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Nov/2017 14:28:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:28:52] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:28:52] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:28:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:28:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:28:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:29:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:29:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2017 14:29:53] "POST /_dash-update-component HTTP/1.1" 200 -
